# Gaussian PRM test

In [ ]:
import pickle
import numpy as np

from swarm_prm.utils.gaussian_prm import GaussianPRM
from swarm_prm.utils.gaussian_utils import GaussianGraphNode
from swarm_prm.envs.roadmap import Roadmap, Obstacle 
from swarm_prm.envs.instance import Instance

In [ ]:

roadmap = Roadmap(200, 200)
safety_radius = 2 
pos = np.array([50, 50])
obstacles  = []
num_samples = 800 

for obs in obstacles:
    roadmap.add_obstacle(obs)
# fig, ax = roadmap.visualize()
start_mean = np.random.rand(2) * 100
goal_mean = np.random.rand(2) * 100

# Agent config
num_agent = 100
agent_radius = 2
sampling_method = "GAUSSIAN_SAMPLING"
#sampling_method = "SWARMPRM_SAMPLING"
hex_radius = 3
TEST_CVAR = False


# Curated test instance
starts = np.array([[70, 50], [70, 150], [130, 50], [130, 150]])
goals = np.array([[25, 50], [25, 150], [175, 50], [175, 150]])

starts_weight = np.array([.5, .5])
goals_weight = np.array([.5, .5])

g_starts = [GaussianGraphNode(start, None, "UNIFORM", radius=10) for start in starts]
g_goals = [GaussianGraphNode(goal, None, "UNIFORM", radius=10) for goal in goals]

instance = Instance(roadmap, g_starts, g_goals)
gaussian_prm = GaussianPRM(instance, num_samples, safety_radius=safety_radius,
                           hex_radius=hex_radius)


gaussian_prm.sample_free_space(sampling_strategy="CVT")
gaussian_prm.build_roadmap(roadmap_method="TRIANGULATION")

gaussian_prm.build_roadmap(roadmap_method="TRIANGULATION", radius=50, collision_check_method="CVAR")
fig, ax = gaussian_prm.visualize_roadmap("test_map")
gaussian_prm.visualize_g_nodes()

with open("solutions/gaussian_prm.pkl", "wb") as f:
    pickle.dump(gaussian_prm, f)


In [ ]:
from scipy.spatial import KDTree

kd_tree = KDTree(gaussian_prm.samples)
kd_tree.query([10, 20])
gaussian_prm.samples[22]
